<a href="https://colab.research.google.com/github/iam-fern/Generate_data/blob/main/ex_sampling_c_100rep_n20_x2_sas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

import statistics as stats
import scipy.stats as ss
import scipy.stats as st
from scipy.stats import f_oneway
from scipy import stats
import scipy.stats as stats
import time
import statsmodels.api as sm

from statsmodels.formula.api import ols

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [4]:
df = pd.read_csv('https://github.com/iam-fern/Research/raw/main/sampling_100rep_n20_x2_sas.csv', index_col=['rep', 'index'])
df

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    39.356822  34.141232  ...  428.841218  1143.973288
     1.0    22.832811  30.970436  ...  323.626146   711.656047
     2.0    17.550017  20.425372  ...  283.812669    80.849304
     3.0    15.944783  47.292068  ...  207.535023   258.864678
     4.0    33.913074  26.372402  ...  413.042488 -1523.105316
...               ...        ...  ...         ...          ...
99.0 15.0   38.589134  31.844129  ...  417.691494  1861.253771
     16.0   17.802384  31.033780  ...  283.468692   -71.703873
     17.0   12.152475  14.318230  ...  237.823140  -248.927749
     18.0   21.735816  11.444608  ...  339.607336   -31.843668
     19.0   26.998311  42.877700  ...  358.099777  1478.142767

[2000 rows x 7 columns]

In [5]:
# ลบข้อมูลที่มี missing values
table = df.dropna()
table

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    39.356822  34.141232  ...  428.841218  1143.973288
     1.0    22.832811  30.970436  ...  323.626146   711.656047
     2.0    17.550017  20.425372  ...  283.812669    80.849304
     3.0    15.944783  47.292068  ...  207.535023   258.864678
     4.0    33.913074  26.372402  ...  413.042488 -1523.105316
...               ...        ...  ...         ...          ...
99.0 15.0   38.589134  31.844129  ...  417.691494  1861.253771
     16.0   17.802384  31.033780  ...  283.468692   -71.703873
     17.0   12.152475  14.318230  ...  237.823140  -248.927749
     18.0   21.735816  11.444608  ...  339.607336   -31.843668
     19.0   26.998311  42.877700  ...  358.099777  1478.142767

[2000 rows x 7 columns]

In [6]:
table.reset_index(inplace=True)
table

,rep,index,x1,x2,x3,e,Y,y_hat,e_hat
0,0.0,0.0,39.356822,34.141232,39.280579,1100.824471,1572.814505,428.841218,1143.973288
1,0.0,1.0,22.832811,30.970436,19.580178,751.403466,1035.282193,323.626146,711.656047
2,0.0,2.0,17.550017,20.425372,24.844378,138.892047,364.661972,283.812669,80.849304
3,0.0,3.0,15.944783,47.292068,49.758903,204.900902,466.399701,207.535023,258.864678
4,0.0,4.0,33.913074,26.372402,24.351898,-1504.917867,-1110.062827,413.042488,-1523.105316
...,...,...,...,...,...,...,...,...,...
1995,99.0,15.0,38.589134,31.844129,44.452775,1811.757025,2278.945265,417.691494,1861.253771
1996,99.0,16.0,17.802384,31.033780,18.619329,-20.912128,211.764819,283.468692,-71.703873
1997,99.0,17.0,12.152475,14.318230,30.518502,-182.466094,-11.104609,237.823140,-248.927749
1998,99.0,18.0,21.735816,11.444608,16.345481,57.615422,307.763669,339.607336,-31.843668


In [8]:
table.set_index(['rep', 'index'], inplace=True)

In [9]:
table

x1         x2  ...       y_hat        e_hat
rep  index                        ...                         
0.0  0.0    39.356822  34.141232  ...  428.841218  1143.973288
     1.0    22.832811  30.970436  ...  323.626146   711.656047
     2.0    17.550017  20.425372  ...  283.812669    80.849304
     3.0    15.944783  47.292068  ...  207.535023   258.864678
     4.0    33.913074  26.372402  ...  413.042488 -1523.105316
...               ...        ...  ...         ...          ...
99.0 15.0   38.589134  31.844129  ...  417.691494  1861.253771
     16.0   17.802384  31.033780  ...  283.468692   -71.703873
     17.0   12.152475  14.318230  ...  237.823140  -248.927749
     18.0   21.735816  11.444608  ...  339.607336   -31.843668
     19.0   26.998311  42.877700  ...  358.099777  1478.142767

[2000 rows x 7 columns]

In [11]:
def get_result(run_replicate):
    new_table = table.loc[run_replicate]

    # show index
    new_table.reset_index(inplace=True)

    new_table.drop(['index'], axis=1, inplace=True)


    # แบ่งข้อมูลออกเป็น 2 กลุ่มด้วย k-mean

    #Set number of clusters
    x = new_table[["x1", "x2", "x3"]]
    k_means = KMeans(n_clusters=2)

    #Run the clustering algorithm
    model = k_means.fit(x)
    model

    #Generate cluster predictions and store in y_kmeans
    y_kmeans = k_means.predict(x)
    y_kmeans

    # kmean_clusters.astype(float)
    kmean_clusters = pd.DataFrame({"clusters" : y_kmeans})

    # เชื่อมข้อมูลทั้ง 2 DataFrame เข้าด้วยกันด้วยคำสั่ง pd.concat
    data_clusters = pd.concat([new_table, kmean_clusters], axis=1)

    data_sort_clusters = data_clusters.sort_values('clusters')

    # print(data_clusters.groupby('clusters').groups)

    data_sort_clusters.reset_index(inplace=True)

    # กลุ่มที่ 1
    data_cluster1 = data_sort_clusters[data_sort_clusters.clusters == 1]

    y = data_cluster1[['Y']]
    x = data_cluster1[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model1 = sm.OLS(y, x).fit()

    DF1 = model1.df_resid

    SSR1 = model1.ssr

    MSE1 = model1.mse_resid

    # กลุ่มที่ 2
    data_cluster0 = data_sort_clusters[data_sort_clusters.clusters == 0]

    y = data_cluster0[['Y']]
    x = data_cluster0[['x1', 'x2', 'x3']]
    x = sm.add_constant(x)
    model0 = sm.OLS(y, x).fit()

    DF0 = model0.df_resid

    SSR0 = model0.ssr

    MSE0 = model0.mse_resid

    # คำนวณค่าสถิติ
    GQ_test_clusters = MSE1 / MSE0

    df1 = DF1
    df0 = DF0

    # คำนวณค่า p-value
    # sf(x, dfn, dfd, loc=0, scale=1) : Survival function (also defined as 1 - cdf, but sf is sometimes more accurate)
    two_tailed = 2.0*(1.0-(ss.f.cdf(GQ_test_clusters, df1, df0)))

    # pdf(x, dfn, dfd, loc=0, scale=1) : Probability density function
    p_greater = ss.f.cdf(GQ_test_clusters, df1, df0)
    
    return df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater

# สร้าง list ของ replicates
list_replicate=list({i[0] for i in table.index})
list_replicate.sort()

# สร้าง list เพื่อเก็บค่า ทุกรอบที่ run
list_df0=[]
list_df1=[]
list_SSR0=[]
list_SSR1=[]
list_MSE0=[]
list_MSE1=[]
list_GQ_test_clusters=[]
list_result_test=[]
list_two_tailed=[]
list_p_greater=[]
list_result_test01=[]

# ใช้ for loop สำหรับการ run ทุก replicates
for run_replicate in list_replicate:
    df0, df1, SSR0, SSR1, MSE0, MSE1, GQ_test_clusters, two_tailed, p_greater=get_result(run_replicate)

    list_df0.append(df0)
    list_df1.append(df1)
    list_SSR0.append(SSR0)
    list_SSR1.append(SSR1)
    list_MSE0.append(MSE0)
    list_MSE1.append(MSE1)
    list_GQ_test_clusters.append(GQ_test_clusters)
    list_two_tailed.append(two_tailed)
    list_p_greater.append(p_greater)

/usr/local/lib/python3.7/dist-packages/statsmodels/regression/linear_model.py:1665: RuntimeWarning: divide by zero encountered in double_scalars
  return self.ssr/self.df_resid


In [12]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1, 
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)
two_tailed_new=two_tailed_new[(two_tailed_new["two_tailed"]<=1) & (two_tailed_new["greater"]<=1)]
two_tailed_new["Result_test"]=two_tailed_new["two_tailed"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                  if p_value < alpha else 'Reject005=1 : Homoscedasticity')
#two_tailed_new_show=two_tailed_new.drop(columns="greater")
two_tailed_new.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed,greater,Result_test
Replicate,,,,,,,,,,
1.0,9.0,3.0,587801.4156490259,13198303.958854029,65311.26840544733,4399434.652951343,67.36103524494412,3.4685447489746224e-06,0.9999982657276255,Reject005=0 : Heteroscedasticity
4.0,6.0,6.0,1323684.7094380166,44089183.10692753,220614.11823966945,7348197.184487921,33.30791901769872,0.0004738737144918126,0.9997630631427541,Reject005=0 : Heteroscedasticity
7.0,8.0,4.0,8979007.8857578,6613025.973296694,1122375.985719725,1653256.4933241736,1.4729970298357915,0.5931096796629522,0.7034451601685239,Reject005=1 : Homoscedasticity
10.0,5.0,7.0,646293.4336238493,7140001.150170631,129258.68672476985,1020000.1643100901,7.8911537023579195,0.03704548187873269,0.9814772590606337,Reject005=0 : Heteroscedasticity
12.0,3.0,9.0,3798656.170049989,34381096.95897994,1266218.7233499964,3820121.884331105,3.0169526116501615,0.3940285316526069,0.8029857341736966,Reject005=1 : Homoscedasticity
14.0,8.0,4.0,2132566.8326338446,20512791.231733825,266570.8540792306,5128197.807933456,19.23765381495625,0.0007272385562107075,0.9996363807218946,Reject005=0 : Heteroscedasticity
16.0,4.0,8.0,135713.658951358,10159642.81721256,33928.4147378395,1269955.35215157,37.43043587401156,0.003384595533494128,0.9983077022332529,Reject005=0 : Heteroscedasticity
18.0,6.0,6.0,2356370.0888731433,7913769.434929029,392728.34814552387,1318961.5724881715,3.358457770406316,0.16605730887843917,0.9169713455607804,Reject005=1 : Homoscedasticity
21.0,4.0,8.0,361654.4371472736,23674608.546644453,90413.6092868184,2959326.0683305566,32.73098034326568,0.004393043229062643,0.9978034783854687,Reject005=0 : Heteroscedasticity


In [13]:
# สร้างตารางผลลัพธ์
alpha = 0.05
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
two_tailed_new=pd.DataFrame({"Replicate":list_replicate, "df0":list_df0, "df1":list_df1, "SSR0":list_SSR0,
                     "SSR1":list_SSR1, "MSE0":list_MSE0, "MSE1":list_MSE1,
                     "GQ_test_clusters":list_GQ_test_clusters, "two_tailed":list_two_tailed, "greater":list_p_greater})
two_tailed_new.set_index(['Replicate'], inplace=True)

two_tailed_new["two_tailed_new"]=two_tailed_new[["two_tailed","greater"]].apply(lambda value: value[0] if value[0]<0.5 else 2*value[1], axis=1)
two_tailed_new["Result_test"]=two_tailed_new["two_tailed_new"].apply(lambda p_value: 'Reject005=0 : Heteroscedasticity' 
                                                                      if p_value < alpha else 'Reject005=1 : Homoscedasticity')


two_tailed_new_show=two_tailed_new.drop(columns=["two_tailed", "greater"])
two_tailed_new_show.astype(str)

,df0,df1,SSR0,SSR1,MSE0,MSE1,GQ_test_clusters,two_tailed_new,Result_test
Replicate,,,,,,,,,
0.0,9.0,3.0,12178206.85381613,908189.9930129009,1353134.0948684588,302729.99767096696,0.22372505342894047,0.24490714065217462,Reject005=1 : Homoscedasticity
1.0,9.0,3.0,587801.4156490259,13198303.958854029,65311.26840544733,4399434.652951343,67.36103524494412,3.4685447489746224e-06,Reject005=0 : Heteroscedasticity
2.0,4.0,8.0,6348820.721096584,6849871.237013961,1587205.180274146,856233.9046267451,0.5394601247954939,0.42425275461257994,Reject005=1 : Homoscedasticity
3.0,7.0,5.0,21920611.001485117,807215.3465461938,3131515.8573550167,161443.06930923875,0.05155428765594659,0.004844926825709698,Reject005=0 : Heteroscedasticity
4.0,6.0,6.0,1323684.7094380166,44089183.10692753,220614.11823966945,7348197.184487921,33.30791901769872,0.0004738737144918126,Reject005=0 : Heteroscedasticity
5.0,8.0,4.0,7857154.277611076,1984307.8138531626,982144.2847013845,496076.95346329064,0.5050958002714643,0.5321122776298561,Reject005=1 : Homoscedasticity
6.0,5.0,7.0,23253154.183761895,1394640.159896295,4650630.836752379,199234.30855661357,0.04284027598624503,0.0006252070849217833,Reject005=0 : Heteroscedasticity
7.0,8.0,4.0,8979007.8857578,6613025.973296694,1122375.985719725,1653256.4933241736,1.4729970298357915,1.4068903203370478,Reject005=1 : Homoscedasticity
8.0,8.0,4.0,20917552.2538173,6842316.615289423,2614694.0317271627,1710579.1538223557,0.654217714602888,0.7195354029971609,Reject005=1 : Homoscedasticity


In [15]:
two_tailed_new_show['Result_test'].value_counts()

Reject005=1 : Homoscedasticity      60
Reject005=0 : Heteroscedasticity    40
Name: Result_test, dtype: int64